In [ ]:
import intake
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import intake_esm
#url = intake_esm.tutorial.get_url('google_cmip6')
#p
url ="https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"

import dask

dask.config.set(scheduler='threads')
dask.config.set(num_workers=20)  # Adjust based on your CPU coresl ="https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"


let us load the data

In [ ]:
cat = intake.open_esm_datastore(url)
cat

In [ ]:
cat.df


let us extract cesm2 OMIP data from the main dataset.

In [ ]:
cesm2 = cat.search(
    activity_id = 'OMIP',
    source_id = 'CESM2',
)
cesm2.df

it seems that there are 2 separate member_ids, let us separate the two.

In [ ]:
cesm2_1111 = cesm2.search(
    member_id = 'r1i1p1f1'
)
cesm2_1111.df

In [ ]:
cesm2_2111 = cesm2.search(
    member_id = 'r2i1p1f1'
)
cesm2_2111.df

In [ ]:
cesm2_1111.unique()['variable_id']

we can see that the variables available in r2 are also in r1, so we will only be using r1 moving forward.

we will also only be using gr, so let us filter that.

In [ ]:
cesm2_filtered = cesm2_1111.search(
    grid_label = 'gr'
)
cesm2_filtered.df

Additionally, all of the table_id Ofx variables are constant, so we will also be filtering those out.

In [ ]:
cesm2_filtered_2 = cesm2_filtered.search(
    table_id = 'Omon'
)
cesm2_filtered_2.df

In [ ]:
varlist = cesm2_filtered_2.unique()['variable_id']

let us now load all the actual data.

In [ ]:
dset_dict = cesm2_filtered_2.to_dataset_dict(
    xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)
dset_dict

In [ ]:
[key for key in dset_dict.keys()][:10]

In [ ]:
data = dset_dict['OMIP.NCAR.CESM2.omip1.Omon.gr']
data

In [ ]:
data.data_vars.values()

In [ ]:
for var in data.data_vars.values():
    print(var.mean(dim=['lat','lon']).groupby('time.year').mean())

In [ ]:
for var in data.data_vars.values():
    try:
        var.isel(lev=0).mean(dim=('lat','lon')).groupby('time.year').mean().plot()
        plt.show()
    except:
        try:
            var.isel(lev_partial=0).mean(dim=('lat','lon')).groupby('time.year').mean().plot()
            plt.show()
        except:
            var.mean(dim=('lat','lon')).groupby('time.year').mean().plot()
            plt.show()